In [63]:
import os
import requests as rq
import pandas as pd
import numpy as np
import time
import json
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select 
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm,trange
from PIL import Image

In [239]:
# 引用其他的Table

with open('cinema.json') as f:               # 引用影城 Id Name
    cinema = json.load(f)
cinemaTable = {}    
for i in range(len(cinema['cinemaId'])):
    cinemaTable[cinema['cinemaName'][i]] = cinema['cinemaId'][i]

movieTable = {}
with open('movie.json') as f:               # 引用電影 Id Name
    movie = json.load(f)
for i in range(len(movie['movieId'])):
    ch = movie['movieName'][i]
    movieTable[movie['movieName'][i].replace(" ","")] = movie['movieId'][i]
    
versionTable = {}
with open('version.json') as f:               # 引用版本 Id Name
    version = json.load(f)
for i in range(len(version['versionId'])):
    versionTable[version['versionName'][i]]= version['versionId'][i]

showing = {'cinemaId':[],'movieId':[],'versionId':[],'showingDate':[],'startTime':[]}

In [240]:
driver = webdriver.Chrome(ChromeDriverManager().install()) # install driver
driver.get("https://www.vscinemas.com.tw/ShowTimes")
select = Select(driver.find_element("name",'CinemaNameTWInfoF'))  # default option
select.select_by_index(1)  # into select 
Info = []
for i in range(0,24): # 要改成range(0,25)
    select = Select(driver.find_element("name",'CinemaNameTWInfoS')) 
    select.select_by_index(i)
        
    cinemaName = select.first_selected_option.text
    cinemaName = re.sub(r'\(.*\)', '', cinemaName).replace(" ","")
    cinemaName = re.sub('[a-zA-Z]','',cinemaName) # 統一名稱
    if cinemaName in cinemaTable.keys():          # 跳過台北松仁
        cinemaId = cinemaTable[cinemaName]
    else:
        print("Jump",cinemaName)
        continue
    time.sleep(2) # for delay
    content  = BeautifulSoup(driver.page_source,"html.parser")
    Time = content.select_one('div #ShowTimeSecondInfo').text.replace(" ","").replace("\n\n","\n").replace('\n'," ").replace("  "," ").replace(" ","\n")
    
    # 把抓到的Time分成需要的格式
    reg = 0    # cal for jump unnecessary value
    count = 0  # cal for jump unnecessary value
    s = ""   # 拼接字串
    movieName = "" # 電影名稱
    showingDate = ""# 播放日期
    startTime = "" # 開始時間
    for ch in Time:
        if count == 2:
            reg = 1
            count = 0
            movieName = ""
        elif reg == 1 and ch != '\n':
            movieName += ch
            count = 0
        elif ch == '\n':
            reg += 1
            count += 1
            if len(s) == 9:
                showingDate = s
            elif len(s) == 5:
                startTime = s
                # 為了對應表可以拿到Id
                name = movieName[movieName.find(')')+1:movieName.rfind('(')]
                # 版本
                if movieName.find('(') > 1:
                    versionName = movieName[:movieName.find(')')]
                else:
                    versionName = movieName[1:movieName.find(')')]
                if versionName in versionTable.keys():
                    versionId = versionTable[versionName]
                else:
                    versionId = ""
                # 電影名稱
                name = re.sub(r'[ ．–·，‧〜:：-]|『.*',"",name)
                if name.find("偶像夢幻祭DREAMLIVE7THTOURALLIEDWORLD") != -1:  # 例外處理 兩邊名稱差太多><
                    name += 'S現場直播'
                if name in movieTable.keys():
                    movieId = movieTable[name]
                else:
                    print(name,"is not in movie table") 
                # 這裡的名稱是用原本的 沒有處理過
                info = versionName +" "+str(versionId)+" "+str(cinemaId)+" "+cinemaName+" "+str(movieId)+" "+movieName[movieName.find(')')+1:movieName.rfind('(')] + " "+showingDate+" "+startTime
                Info.append(info)
                showing['cinemaId'].append(cinemaId)
                showing['movieId'].append(movieId)
                showing['showingDate'].append(showingDate)
                showing['versionId'].append(versionId)
                showing['startTime'].append(startTime)
            s = ""
        else:
            s += ch
            count = 0
    

C:\Users\User\AppData\Local\Temp\ipykernel_2668\3516825496.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install()) # install driver


Jump 台北松仁威秀
Jump 台北松仁威秀
2001太空漫遊 is not in movie table


In [243]:
showing_CSV = pd.DataFrame(showing)
showing_CSV.to_csv("showing.csv",index=False,encoding="utf_8_sig")

In [241]:
for i in Info:
    print(i)
prin

4DX3D 12 1 台北信義威秀影城 6168 黑豹2：瓦干達萬歲 11月13日星期日 10:00
4DX3D 12 1 台北信義威秀影城 6168 黑豹2：瓦干達萬歲 11月13日星期日 16:00
4DX3D 12 1 台北信義威秀影城 6168 黑豹2：瓦干達萬歲 11月13日星期日 19:00
4DX3D 12 1 台北信義威秀影城 6168 黑豹2：瓦干達萬歲 11月13日星期日 01:00
4DX3D 12 1 台北信義威秀影城 6168 黑豹2：瓦干達萬歲 11月14日星期一 10:00
4DX3D 12 1 台北信義威秀影城 6168 黑豹2：瓦干達萬歲 11月14日星期一 16:00
4DX3D 12 1 台北信義威秀影城 6168 黑豹2：瓦干達萬歲 11月14日星期一 19:00
4DX3D 12 1 台北信義威秀影城 6168 黑豹2：瓦干達萬歲 11月14日星期一 01:00
4DX3D 12 1 台北信義威秀影城 6168 黑豹2：瓦干達萬歲 11月15日星期二 10:00
4DX3D 12 1 台北信義威秀影城 6168 黑豹2：瓦干達萬歲 11月15日星期二 16:00
4DX3D 12 1 台北信義威秀影城 6168 黑豹2：瓦干達萬歲 11月15日星期二 19:00
4DX3D 12 1 台北信義威秀影城 6168 黑豹2：瓦干達萬歲 11月15日星期二 01:00
4DX3D 12 1 台北信義威秀影城 6168 黑豹2：瓦干達萬歲 11月16日星期三 10:00
4DX3D 12 1 台北信義威秀影城 6168 黑豹2：瓦干達萬歲 11月16日星期三 16:00
4DX3D 12 1 台北信義威秀影城 6168 黑豹2：瓦干達萬歲 11月16日星期三 19:00
4DX3D 12 1 台北信義威秀影城 6168 黑豹2：瓦干達萬歲 11月16日星期三 01:00
4DX3D 12 1 台北信義威秀影城 6168 黑豹2：瓦干達萬歲 11月17日星期四 10:00
4DX3D 12 1 台北信義威秀影城 6168 黑豹2：瓦干達萬歲 11月17日星期四 16:00
4DX3D 12 1 台北信義威秀影城 6168 黑豹2：瓦干達萬歲 11月17日星期四 19:00
4DX3D 12 1 台北信義威秀影城 6168 黑豹2：瓦干

NameError: name 'prin' is not defined